# Reddit Scraper für vergangene Daten 

## Import 

In [20]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch


In [2]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [3]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [4]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Kraken Pay AMA with Jared Blake, $5000 USDG* in prizes up for grabs!, Score: 32
Title: Daily Crypto Discussion - February 4, 2025 (GMT+0), Score: 28
Title: Someone Just Moved 50 Bitcoin (BTC) Mined 15 Years Ago - HODL'ed from $0.10 to $100K. Who of You Just Got Out of Prison?, Score: 344
Title: Somethings wrong I can feel it, Score: 892
Title: China Counters Trump, Imposes 15% Tariff On Coal, Gas Imports From US, Score: 353


Cryptos und Subreddits 

In [5]:
cryptos = [
    ("Ethereum", "ETH"),
    ("Solana", "SOL"),
    ("Avalanche", "AVAX"),
    ("Polkadot", "DOT"),
    ("Near Protocol", "NEAR"),
    ("Polygon", "MATIC"),
    ("XRP", "XRP"),
    ("Cardano", "ADA"),
    ("Chainlink", "LINK"),
    ("PYR", "PYR"),
    ("Cronos", "CRO"),
    ("Chiliz", "CHZ"),
    ("Illuvium", "ILV"),
    ("Ronin", "RON"),
    ("Band Protocol", "BAND"),
    ("Opulous", "OPUL"),
    ("Tia", "TIA"),
    ("Numeraire", "NMR"),
    ("Athens", "ATH"),
    ("Sui", "SUI"),  # Dein gewünschtes SUI-Token
    ("Binance Coin", "BNB"),
    ("Dogecoin", "DOGE"),
    ("Litecoin", "LTC"),
    ("Uniswap", "UNI"),
    ("Cosmos", "ATOM"),
    ("Aave", "AAVE"),
    ("The Graph", "GRT"),
    ("Decentraland", "MANA"),
    ("Sandbox", "SAND"),
    ("Axie Infinity", "AXS"),
    ("Render Token", "RNDR"),
    ("Fantom", "FTM"),
    ("Helium", "HNT"),
    ("Gala", "GALA"),
    ("Theta Network", "THETA"),
    ("Kava", "KAVA"),
    ("Stacks", "STX"),
    ("PancakeSwap", "CAKE"),
    ("1inch", "1INCH"),
    ("Enjin Coin", "ENJ"),
    ("Curve DAO", "CRV"),
    ("Convex Finance", "CVX"),
    ("Flow", "FLOW"),
    ("Arweave", "AR"),
    ("Celo", "CELO"),
    ("Immutable X", "IMX"),
    ("dYdX", "DYDX"),
    ("Lido DAO", "LDO"),
    ("Rocket Pool", "RPL"),
    ("Algorand", "ALGO"),
]


In [6]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scraping 

Scraping Funktionen

In [7]:
# Funktion, um Posts und Kommentare gezielt zu Kryptowährungen zu scrapen
def scrape_reddit(start_date, end_date, mode="initial"):
    start_timestamp = int(start_date.timestamp())  # Umwandlung in Unix-Zeit
    end_timestamp = int(end_date.timestamp())  # Umwandlung in Unix-Zeit

    posts = []
    comments = []
    post_ids = set()  # Vermeidung doppelter Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche nach {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche gezielt nach dem Namen und Symbol der Kryptowährung
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Post gefunden: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# Einmaliger Scrape für die letzten 3 Monate
start_of_period = datetime(2024, 11, 1)  # 1. November 2024 als `datetime`
now = datetime.now()  # Aktuelle Zeit als `datetime`
print("Starte den einmaligen Scrape für die letzten 3 Monate...")
df_posts_initial, df_comments_initial = scrape_reddit(start_of_period, now, mode="initial")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den einmaligen Scrape für die letzten 3 Monate...
Suche nach Ethereum (ETH) in r/CryptoCurrency...
Post gefunden: For every action there is an equal and opposite reaction. (Suchbegriff: Ethereum)
Post gefunden: Ethereum leads crypto liquidation spree with $617M losses in 24 hours (Suchbegriff: Ethereum)
Post gefunden: QR coin update, rumors of Musk move into quantum computing (Suchbegriff: Ethereum)
Post gefunden: [Technical Discussion] How do AI Agents+Crypto actually work? Virtuals protcol etc. (Suchbegriff: Ethereum)
Post gefunden: Ethereum Price Tanks 25%: What’s Next After the Major Decline? (Suchbegriff: Ethereum)
Post gefunden: Bitcoin (BTC) & Ethereum (ETH) Exchange Liquidation Map: There Is Nothing Left to Sell Off... (Suchbegriff: Ethereum)
Post gefunden: The ETH to BTC Ratio just flash crashed to .02337, a lows not seen since January 2021. (Suchbegriff: Ethereum)
Post gefunden: Free Fall: Dow Futures Slide, Crypto Liquidations Top $1B (Suchbegriff: Ethereum)
Post gefu

## Clean

In [28]:
def clean_data(df_posts, df_comments, comment_threshold=500):# Anpassbarer Schwellenwert für Kommentare pro Nutzer
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])

    # 4. Datum & Uhrzeit in separate Spalten aufteilen (Daten normalisieren)
    df_posts["date"] = df_posts["created_utc"].dt.date  # YYYY-MM-DD
    df_posts["time"] = df_posts["created_utc"].dt.time  # HH:MM:SS

    df_comments["date"] = df_comments["created_utc"].dt.date
    df_comments["time"] = df_comments["created_utc"].dt.time

    # 5. Original `created_utc`-Spalte entfernen
    df_posts.drop(columns=["created_utc"], inplace=True)
    df_comments.drop(columns=["created_utc"], inplace=True)

    # 6. Filterung nach Qualität (Spam oder irrelevante Daten entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen

    # 7. Entferne bekannte Bot-Accounts
    bot_accounts = ["AutoModerator", "coinfeeds-bot", "devCheckingIn"]
    df_comments = df_comments[~df_comments["author"].isin(bot_accounts)]

    # 8. Entferne Nutzer mit übermäßigen Kommentaren
    comment_counts = df_comments["author"].value_counts()
    frequent_users = comment_counts[comment_counts > comment_threshold].index  # Nutzer über Grenze
    df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    print(f"✅ Daten bereinigt: {df_comments.shape[0]} Kommentare übrig (nach Spam-Filter).")

    return df_posts, df_comments


In [27]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts, df_comments, comment_threshold=300)# Anpassbarer Schwellenwert für Kommentare pro Nutzer


# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

✅ Daten bereinigt: 130578 Kommentare übrig (nach Spam-Filter).
Bereinigte Posts: 2269
Bereinigte Kommentare: 130578


## Model fuer das Sentiment 

In [50]:
# Neues Modell laden
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 🔹 Sicherstellen, dass die Spalte "body" existiert
if "body" not in df_comments_clean.columns:
    raise ValueError("Fehler: Die CSV-Datei enthält keine 'body'-Spalte mit Kommentaren!")

# 🔹 Funktion zur Sentiment-Analyse
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0  # Leere Kommentare sind neutral

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = F.softmax(outputs.logits, dim=1)[0]
    labels = ["negative", "neutral", "positive"]
    sentiment = labels[torch.argmax(scores).item()]
    confidence = scores.max().item()

    return sentiment, confidence

# 🔹 Sentiment für alle Kommentare berechnen
df_comments_clean["sentiment"], df_comments_clean["sentiment_confidence"] = zip(*df_comments_clean["body"].map(analyze_sentiment))

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

c:\Users\hp\anaconda3\envs\dennis\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [38]:
df_comments_clean.head() 

,post_id,comment_id,author,score,body,date,time,sentiment,sentiment_confidence
0,1ihc9fm,mavxre2,Equivalent-Fan-1362,1,I’m heavy in ltc and I just buy and look forwa...,2025-02-04,07:23:17,negative,0.657665
1,1ihc9fm,maw66ik,Traditional-Fan-9315,1,- Sir Isaac Bitcoin,2025-02-04,08:48:07,positive,0.918842
2,1ihc9fm,maw29y8,foxroadblue,2,LTC is a very nice example of what can happen ...,2025-02-04,08:07:40,positive,0.880970
3,1ihc9fm,maw4gvb,Equivalent-Fan-1362,1,What’s wrong with ltc? It still has a compelli...,2025-02-04,08:30:17,positive,0.827299
5,1igvb78,mast4yh,callmev269,42,The one thing that ETH is ahead these days...,2025-02-03,20:34:56,positive,0.821829


## Export 

In [39]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/reddit"

Funktion zum Export 

In [55]:
def append_to_csv(df_new, filename):
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls die Datei bereits existiert, lese sie ein und hänge die neuen Daten an
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep=";", encoding="utf-8-sig", on_bad_lines="skip")
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new  # Falls die Datei nicht existiert, erstelle sie neu

        # CSV speichern mit besserer Formatierung für Excel
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",  # Tabulator als Trennzeichen für bessere Excel-Kompatibilität
            encoding="utf-8-sig",  # Richtige Zeichenkodierung für Excel
            lineterminator="\n"  # Zeilenumbrüche korrekt speichern
        )
        print(f"Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments):
    try:
        # Posts an bestehende Datei anhängen
        append_to_csv(df_posts, "reddit_posts.csv")
        # Kommentare an bestehende Datei anhängen
        append_to_csv(df_comments, "reddit_comments.csv")

    except Exception as e:
        print(f"Fehler beim Export: {e}")


In [56]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)

Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit\reddit_posts.csv
Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit\reddit_comments.csv
